In [126]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [127]:
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

---

#### Read reference data into dataframe

In [232]:
df = pd.read_csv('../reference_w_loc.csv')

In [233]:
df.head(3)

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,state,country
0,0,"Man repts. witnessing &quot;flash, followed by...",NaN,"Iowa City, IA",19951009,unknown,19951009,41.661256,-91.529911,Iowa City,Iowa,us
1,1,"Man on Hwy 43 SW of Milwaukee sees large, bri...",2 min.,"Milwaukee, WI",19951011,unknown,19951010,43.034993,-87.922497,Milwaukee,Wisconsin,us
2,2,Telephoned Report:CA woman visiting daughter w...,NaN,"Shelton, WA",19950103,unknown,19950101,47.215094,-123.100707,Shelton,Washington,us


#### There are 138 rows with no sighted_at time information. We replace this date with their respective reported_at information

In [234]:
df[df['sighted_at'] == 0].shape

(138, 12)

In [235]:
# replace dates without a sighted_at with their reported_at value
for index,row in df[df['sighted_at'] == 0].iterrows():
    df.ix[index, 'sighted_at'] = row['reported_at']

In [236]:
# as expected there are no more missing sighted_at values
df[df['sighted_at'] == 0].shape

(0, 12)

#### Extract year from sighted_at and add as column

In [237]:
# add column 'year'
df['year'] = 0

In [238]:
def custom_round(x, base=10):
    return int(base * round(float(x)/base))

In [239]:
for index, row in df.iterrows():
    row_date = str(row['sighted_at'])
    df.ix[index, 'year'] = custom_round(datetime.strptime(row_date, '%Y%m%d').year)

In [240]:
df.sample(5)

,id,description,duration,location,reported_at,shape,sighted_at,geocoded_latitude,geocoded_longitude,city,state,country,year
11055,17105,Observed what appeared to be a dim satellite m...,15 Seconds,"Portland, OR",20021018,light,20021016,45.520247,-122.674195,Portland,Oregon,us,2000
10543,16208,Low flying triangular formation? of silent bl...,1 to 2 minutes,"Hampton, VA",20020525,formation,19730615,37.030097,-76.345206,Hampton City,Virginia,us,1970
2670,3510,"after almost causing a helicoptor to crash,a s...",45 min's,"Topeka, KS",19990302,disk,19930522,39.049011,-95.677556,Topeka,Kansas,us,1990
36462,57970,7 dim lights triangular formation 30 sec easte...,30 sec,"Southampton, NY",20100108,formation,20100107,40.884267,-72.389530,Southampton,New York,us,2010
23468,37698,"triangle w/3 lights (red, green, and white) on...",10 seconds,"Lapeer, MI",20060816,triangle,20060816,43.051416,-83.318834,Lapeer,Michigan,us,2010


---

#### Read in population csv

In [241]:
pop_df = pd.read_csv('pop_final.csv')

In [242]:
pop_df.dtypes

State         object
Year           int64
Population    object
dtype: object

In [243]:
pop_df = pop_df.rename(index=str, columns={"State": "state", "Year": "year", "Population": "population"})
pop_df = pop_df.drop_duplicates()

In [244]:
pop_df.sample(4)

,state,year,population
1364,New Jersey,2010,8791894
1812,South Carolina,1990,3486703
914,Maryland,2010,5773552
856,Maine,1880,648936


In [245]:
df_merged = df.merge(pop_df, on=['state','year'], how='left')

In [247]:
df_merged[df_merged['population'].isnull()].shape

(72, 14)

In [251]:
df_merged = df_merged[df_merged['population'].isnull() == False]

In [255]:
df.shape, df_merged.shape

((38511, 13), (38439, 14))

"puerto rico" is not in the population dataset so those rows were NaN and removed

At this point, we've added the population column

#### save df to pick / read df from pickle

In [254]:
# df.to_pickle('../pickle_files/df_ref_w_population.pkl')
# df = pd.read_pickle('../pickle_files/df_ref_w_population.pkl')

----

In [252]:
df.to_csv('reference_w_population.csv', index=False)